
# Воспроизведение "King James Programming" для русского языка, или "Ветхий алгоритм"

http://kingjamesprogramming.tumblr.com/

## Данные

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import regex 
import numpy as np

In [3]:
# Читаем алгоритмы Кормена
with open('./texts/cormen.txt', encoding='utf8') as f:
    cormen = f.read().replace(u'\xa0', u' ').replace(u'\x0c', u'.').replace(u'\xad', '')
    cormen = cormen.replace(".w", "м").replace("19", "ю").replace("ё", "е")
cormen = regex.sub(u'[^\p{L}\ \n\.\!\?\;]', '', cormen) # оставляем только буквы и знаки препенания
cormen = regex.sub('[\!\?\;]', '.', cormen) # заменяем все знаки препинания на точки
cormen = regex.sub('Глава', '', cormen)
cormen = regex.sub('\n+\ *\n*\ *', ' ', cormen) # сокращаем \n
cormen = regex.sub('\.+[ *\.*\n*]*', '. ', cormen) # сокращаем точки

In [4]:
import nltk
from nltk.tokenize import TweetTokenizer

sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
tokenizer = TweetTokenizer()

cormen_sentences = []

for sent in sent_detector.tokenize(cormen.lower()):
    tokens = tokenizer.tokenize(sent)
    if tokens[0].isdigit():
        tokens = tokens[1:]
    cormen_sentences.append(tokens)

In [5]:
# Читаем книгу про юникс
with open('./texts/unix.txt', encoding='utf8') as f:
    unix = f.read().replace(u'\xa0', u' ').replace(u'\x0c', u'.').replace('\xad', '')
    unix = unix.replace(".w", "м").replace("19", "ю").replace("ё", "е")

unix = regex.sub(u'[^\p{L}\ \n\.\!\?\;]', '', unix) # оставляем только буквы и знаки препенания
unix = regex.sub('[\!\?\;]', '.', unix) # заменяем все знаки препинания на точки
unix = regex.sub('Глава', '', unix)
unix = regex.sub('\n+\ *\n*\ *', ' ', unix) # сокращаем \n
unix = regex.sub('\.+[ *\.*\n*]*', '. ', unix) # сокращаем точки


In [6]:
unix_sentences = []

for sent in sent_detector.tokenize(unix.lower()):
    tokens = tokenizer.tokenize(sent)
    if tokens[0].isdigit():
        tokens = tokens[1:]
    unix_sentences.append(tokens)

In [7]:
# читаем ветхий завет
with open('./texts/testament.txt', encoding='utf8') as f:
    testament = f.read().replace(u'\xa0', u' ').replace(u'\x0c', u'.').replace('\xad', '')
    testament = testament.replace(".w", "м").replace("19", "ю").replace("ё", "е")
# # чистим ветхий завет
testament = regex.sub(u'[^\p{L}\ \n\.\!\?\;]', '', testament) # оставляем только буквы и знаки препенания
testament = regex.sub('[\!\?\;]', '.', testament) # заменяем все знаки препинания на точки
testament = regex.sub('Глава', '', testament)
testament = regex.sub('\n+\ *\n*\ *', ' ', testament) # сокращаем \n
testament = regex.sub('\.+[ *\.*\n*]*', '. ', testament) # сокращаем точки

In [8]:
testament_sentences = []

for sent in sent_detector.tokenize(testament.lower()):
    tokens = tokenizer.tokenize(sent)
    if tokens[0].isdigit():
        tokens = tokens[1:]
    testament_sentences.append(tokens)

### Подготовка данных

Оставим предложения длиной не меньше 5.

In [10]:
code_sentences = cormen_sentences + unix_sentences
code_sentences = [s for s in code_sentences if 5 <= len(s)]
testament_sentences = [s for s in testament_sentences if 5 <= len(s)]

In [11]:
all_words = [w for s in testament_sentences for w in s] + [w for s in code_sentences for w in s]
words = np.array(list(set(all_words)))

In [12]:
# np.save("words_alph.npy", words)
words = np.load("words_alph.npy")

Обучим модель fasttext для представлений слов.

In [14]:
import gensim
from gensim.models import FastText

In [13]:
np.random.seed(923418)
all_text = np.array(testament_sentences + code_sentences)
np.random.shuffle(all_text)

In [14]:
all_text = [" ".join(s).lower().split(" ") for s in all_text]

In [18]:
fast_text_model = FastText(size=256, window=4, min_count=2)
fast_text_model.build_vocab(sentences=all_text)
fast_text_model.train(sentences=all_text, total_examples=len(all_text), epochs=10)

In [13]:
from gensim.test.utils import get_tmpfile

fname = get_tmpfile("fasttext.model")

# fast_text_model.save(fname)


In [16]:
# fast_text_model.save("fast_text_model.model")

In [15]:
fast_text_model = FastText.load(fname)

Создаем обучающую и тестовую выборки и алфавит всего текста.

In [16]:
np.random.seed(923418)
idx = np.random.choice(range(len(code_sentences)), len(code_sentences) // 10, False)
mask = np.ones((len(code_sentences)), dtype=bool)
mask[idx] = False

In [17]:
code_train = np.array(code_sentences)[mask]
code_test = np.array(code_sentences)[~mask]

In [18]:
idx = np.random.choice(range(len(testament_sentences)), len(testament_sentences) // 10, False)
mask = np.ones((len(testament_sentences)), dtype=bool)
mask[idx] = False

In [19]:
testament_train = np.array(testament_sentences)[mask]
testament_test = np.array(testament_sentences)[~mask]

In [20]:
alphabet = " ".join(words)

In [25]:
alphabet = sorted(set(list(alphabet)))
print(alphabet)

[' ', '.', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я']


In [22]:
char_idx = dict(zip(alphabet, np.eye(len(alphabet))))

### Импорт и кофиг tensorflow и keras

In [26]:
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau
from tensorflow.keras.layers import (Bidirectional, TimeDistributed, LSTM, Activation, Dense, Dropout, Flatten, Input, MaxPool2D, Embedding)
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.losses import categorical_crossentropy, sparse_categorical_crossentropy
from tensorflow.keras.layers import Layer
from tensorflow.keras.utils import Sequence
from tensorflow.keras.backend import set_session
import tensorflow as tf

In [27]:
config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.4
config.gpu_options.visible_device_list='0'
config.gpu_options.allow_growth = True
# config.log_device_placement=True
sess = tf.Session(config=config)
set_session(sess)

### LSTM на словах с cosine loss

In [21]:
class generator(Sequence):
    
    def __init__(self, code, bible, batch_size=100, window=5):
        self.text = np.concatenate([np.array(bible), np.array(code)])
        np.random.shuffle(self.text)
        self.batch_size = batch_size
        self.window = window
        self.indices = np.arange(self.text.shape[0])
        
    def __len__(self):
        return len(self.text) // self.batch_size
    
    def __getitem__(self, idx):
        inds = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        text = self.text[inds]
        text = [w for s in text for w in s]
        text_vecs = [fast_text_model.wv[w] for w in text]
        batch_in = []
        batch_out = []
        for i in range(self.window, len(text_vecs) - self.window):
            batch_in.append(text_vecs[i - self.window:i])  
            batch_out.append(text_vecs[i])
        return np.array(batch_in), np.array(batch_out)
    
    def on_epoch_end(self):
        np.random.shuffle(self.indices)

In [22]:
def get_model(dropout=0.3, window=5):
    model = Sequential()
    model.add(Bidirectional(LSTM(256, return_sequences=True), input_shape=(window, 256)))
    if dropout > 0:
        model.add(Dropout(dropout))
        
    model.add(LSTM(256))
    return model

In [24]:
m = get_model()

In [25]:
m.compile(loss='cosine_proximity', optimizer=Adam(), metrics=['CosineSimilarity'])

In [26]:
cos_his = m.fit_generator(generator=generator(code_train, testament_train, batch_size=200), 
                validation_data=generator(code_test, testament_test, batch_size=200),
                workers=16, use_multiprocessing=True, 
                epochs=800, shuffle=False)

W1009 19:31:18.308707 140395259537216 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/800
217/217 [==============================] - 15s 70ms/step - loss: -0.4608 - cosine_similarity: -0.4610 - val_loss: -0.4993 - val_cosine_similarity: -0.4993
Epoch 2/800
217/217 [==============================] - 13s 59ms/step - loss: -0.5014 - cosine_similarity: -0.5014 - val_loss: -0.5137 - val_cosine_similarity: -0.5137
Epoch 3/800
217/217 [==============================] - 13s 60ms/step - loss: -0.5132 - cosine_similarity: -0.5132 - val_loss: -0.5222 - val_cosine_similarity: -0.5222
Epoch 4/800
217/217 [==============================] - 14s 64ms/step - loss: -0.5213 - cosine_similarity: -0.5212 - val_loss: -0.5289 - val_cosine_similarity: -0.5289
Epoch 5/800
217/217 [==============================] - 13s 60ms/step - loss: -0.5275 - cosine_similarity: -0.5275 - val_loss: -0.5334 - val_cosine_similarity: -0.5333
Epoch 6/800
217/217 [==============================] - 13s 60ms/step - loss: -0.5328 - cosine_similarity: -0.5328 - val_loss: -0.5367 - val_cosine_similarity: -0.536

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



217/217 [==============================] - 15s 71ms/step - loss: -0.6394 - cosine_similarity: -0.6394 - val_loss: -0.5695 - val_cosine_similarity: -0.5695
Epoch 87/800
217/217 [==============================] - 16s 73ms/step - loss: -0.6397 - cosine_similarity: -0.6398 - val_loss: -0.5689 - val_cosine_similarity: -0.5689
Epoch 88/800
217/217 [==============================] - 16s 72ms/step - loss: -0.6401 - cosine_similarity: -0.6402 - val_loss: -0.5688 - val_cosine_similarity: -0.5688
Epoch 89/800
217/217 [==============================] - 16s 72ms/step - loss: -0.6407 - cosine_similarity: -0.6407 - val_loss: -0.5691 - val_cosine_similarity: -0.5691
Epoch 90/800
217/217 [==============================] - 16s 72ms/step - loss: -0.6410 - cosine_similarity: -0.6410 - val_loss: -0.5692 - val_cosine_similarity: -0.5692
Epoch 91/800
217/217 [==============================] - 17s 76ms/step - loss: -0.6414 - cosine_similarity: -0.6414 - val_loss: -0.5689 - val_cosine_similarity: -0.5689
Epoch

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



217/217 [==============================] - 16s 76ms/step - loss: -0.6921 - cosine_similarity: -0.6921 - val_loss: -0.5729 - val_cosine_similarity: -0.5728
Epoch 646/800
217/217 [==============================] - 17s 77ms/step - loss: -0.6921 - cosine_similarity: -0.6921 - val_loss: -0.5730 - val_cosine_similarity: -0.5730
Epoch 647/800
217/217 [==============================] - 17s 77ms/step - loss: -0.6922 - cosine_similarity: -0.6923 - val_loss: -0.5731 - val_cosine_similarity: -0.5731
Epoch 648/800
217/217 [==============================] - 18s 82ms/step - loss: -0.6922 - cosine_similarity: -0.6922 - val_loss: -0.5728 - val_cosine_similarity: -0.5728
Epoch 649/800
217/217 [==============================] - 16s 73ms/step - loss: -0.6924 - cosine_similarity: -0.6925 - val_loss: -0.5726 - val_cosine_similarity: -0.5727
Epoch 650/800
217/217 [==============================] - 16s 73ms/step - loss: -0.6924 - cosine_similarity: -0.6924 - val_loss: -0.5732 - val_cosine_similarity: -0.5733


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



217/217 [==============================] - 16s 73ms/step - loss: -0.6944 - cosine_similarity: -0.6945 - val_loss: -0.5735 - val_cosine_similarity: -0.5736
Epoch 736/800
217/217 [==============================] - 16s 73ms/step - loss: -0.6942 - cosine_similarity: -0.6942 - val_loss: -0.5728 - val_cosine_similarity: -0.5729
Epoch 737/800
217/217 [==============================] - 16s 73ms/step - loss: -0.6947 - cosine_similarity: -0.6947 - val_loss: -0.5739 - val_cosine_similarity: -0.5740
Epoch 738/800
217/217 [==============================] - 16s 73ms/step - loss: -0.6944 - cosine_similarity: -0.6945 - val_loss: -0.5729 - val_cosine_similarity: -0.5731
Epoch 739/800
217/217 [==============================] - 16s 72ms/step - loss: -0.6942 - cosine_similarity: -0.6942 - val_loss: -0.5724 - val_cosine_similarity: -0.5725
Epoch 740/800
217/217 [==============================] - 16s 74ms/step - loss: -0.6939 - cosine_similarity: -0.6939 - val_loss: -0.5734 - val_cosine_similarity: -0.5736


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [27]:
m.save('./models/cos.h5')

### LSTM на словах с cross-entropy loss

In [34]:
class generator(Sequence):
    
    def __init__(self, code, bible, batch_size=100, window=5):
        self.text = np.concatenate([np.array(bible), np.array(code)])
        np.random.shuffle(self.text)
        self.batch_size = batch_size
        self.window = window
        self.indices = np.arange(self.text.shape[0])
        
    def __len__(self):
        return len(self.text) // self.batch_size
    
    def __getitem__(self, idx):
        inds = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        text = self.text[inds]
        text = [w for s in text for w in s]
        text_vecs = [fast_text_model.wv[w] for w in text]
        batch_in = []
        batch_out = []
        for i in range(self.window, len(text_vecs) - self.window):
            batch_in.append(text_vecs[i - self.window:i])  
            batch_out.append(np.where(words==text[i])[0][0])
        return np.array(batch_in), np.array(batch_out)
    
    def on_epoch_end(self):
        np.random.shuffle(self.indices)

In [35]:
def get_model(dropout=0.3, window=5):
    model = Sequential()
    model.add(Bidirectional(LSTM(256, return_sequences=True),input_shape=(window, 256)))
    if dropout > 0:
        model.add(Dropout(dropout))
        
    model.add(LSTM(256))
    if dropout > 0:
        model.add(Dropout(dropout))
    model.add(Dense(len(words)))
    model.add(Activation('softmax'))
    return model

In [36]:
m = get_model()

In [37]:
m.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['CosineSimilarity'])

In [ ]:
ce_his = m.fit_generator(generator=generator(code_train, testament_train, batch_size=200), 
                validation_data=generator(code_test, testament_test, batch_size=200),
                workers=16, use_multiprocessing=True, 
                epochs=350, shuffle=False,)

In [35]:
m.save("./models/ce.h5")

### LSTM на символах с cross-entropy loss

In [ ]:
class generator(Sequence):
    
    def __init__(self, code, bible, batch_size=100, window=30):

        self.text = np.concatenate([np.array(bible), np.array(code)])
        np.random.shuffle(self.text)
        self.batch_size = batch_size
        self.window = window
        self.indices = np.arange(self.text.shape[0])
        
        
    def __len__(self):
        return len(self.text) // self.batch_size
    
    def __getitem__(self, idx):
        inds = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        text = self.text[inds]
        text = " ".join([w for s in text for w in s])
        batch_in = []
        batch_out = []
        for i in range(self.window, len(text) - self.window):
            batch_in.append([char_idx[ch] for ch in text[i - self.window:i]])
            batch_out.append(char_idx[text[i]])
        return np.array(batch_in), np.array(batch_out)
    
    def on_epoch_end(self):
        np.random.shuffle(self.indices)

In [ ]:
from tensorflow.keras.layers import TimeDistributed
def get_model(dropout=0.5, window=30):
    model = Sequential()
    model.add(LSTM(256, return_sequences=True, input_shape=(window, len(alphabet))))
    if dropout > 0:
        model.add(Dropout(dropout))
        
    model.add(LSTM(256))
    if dropout > 0:
        model.add(Dropout(dropout))
    model.add(Dense(len(alphabet)))
    model.add(Activation('softmax'))

    return model

In [ ]:
m = get_model()

In [ ]:
m.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [ ]:
char_his = m.fit_generator(generator=generator(code_train, testament_train, batch_size=40), 
                validation_data=generator(code_test, testament_test, batch_size=40),
                workers=16, use_multiprocessing=True, 
                epochs=50, shuffle=False,)

In [ ]:
m.save("./models/char_lstm_under.h5")

### Bidirectional LSTM на символах с cross-entropy loss

In [25]:
class generator(Sequence):
    
    def __init__(self, code, bible, batch_size=100, window=30):

        self.text = np.concatenate([np.array(bible), np.array(code)])
        np.random.shuffle(self.text)
        self.batch_size = batch_size
        self.window = window
        self.indices = np.arange(self.text.shape[0])
        
        
    def __len__(self):
        return len(self.text) // self.batch_size
    
    def __getitem__(self, idx):
        inds = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        text = self.text[inds]
        text = " ".join([w for s in text for w in s])
        batch_in = []
        batch_out = []
        for i in range(self.window, len(text) - self.window):
            batch_in.append([char_idx[ch] for ch in text[i - self.window:i]])
            batch_out.append(char_idx[text[i]])
        return np.array(batch_in), np.array(batch_out)
    
    def on_epoch_end(self):
        np.random.shuffle(self.indices)

In [26]:
from tensorflow.keras.layers import TimeDistributed
def get_model(dropout=0.5, window=30):
    model = Sequential()
    model.add(Bidirectional(LSTM(256, return_sequences=True),input_shape=(window, len(alphabet))))
    if dropout > 0:
        model.add(Dropout(dropout))
        
    model.add(LSTM(256))
    if dropout > 0:
        model.add(Dropout(dropout))
    model.add(Dense(len(alphabet)))
    model.add(Activation('softmax'))
    
    return model

In [41]:
m = get_model()

In [42]:
m.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [ ]:
ce1_his = m.fit_generator(generator=generator(code_train, testament_train, batch_size=40), 
                validation_data=generator(code_test, testament_test, batch_size=40),
                workers=16, use_multiprocessing=True, 
                epochs=30, shuffle=False)

In [44]:
m.save("./models/char_bidirect_under.h5")

### Инференс

In [28]:
char_models = [
    "./models/char_lstm.h5",
    "./models/char_bidirect.h5",
    "./models/char_bidirect_under.h5"
    "./models/char_lstm_under.h5"
]
words_models = [
    "./models/ce.h5",
    "./models/cos.h5",
]

Функция sample семплирует объект с некоторой вероятностью, изменяемой температурой. Температуру мы будем использовать, чтобы разнообразить предсказания моделей.

In [31]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / (temperature + 1e-2)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def pformat(seq):
    if not type(seq) is str:
        seq = " ".join(seq)
    return seq

def infer(model, mode="char", window=30, divs=np.linspace(0, 1, 6), seq_len=200):
    '''
    Returns sequences of length seq_len for each diversity
    mode: {"char", "words_ce", "words_cos"}
    '''
    vocab = alphabet if mode == "char" else words
    sentence = np.random.choice(np.concatenate([code_test, testament_test]), 10)
    
    if mode == "char":
        sentence = " ".join([" ".join(s) for s in sentence])
        sentence = sentence[:window]
        start = [char_idx[ch] for ch in sentence]
    elif mode == "words_cos" or "words_ce":
        sentence = [w for s in sentence for w in s][:window]
        start = [fast_text_model.wv[w] for w in sentence]
    
    for diversity in divs:
        print(f'----- Diversity: {diversity}\n')
        print(f'----- Generating with seed:\n"{pformat(sentence)}"\n')
        res = '' if mode == "char" else []
        if seq_len is 200:
            seq_len = 200 if mode == "char" else 40
        for i in range(seq_len):
            
            preds = model.predict(np.array(start)[None, :, :], verbose=0)[0]
            
            # sample next word
            if mode == "char" or mode == "words_ce":
                next_index = sample(preds, diversity)
                next_word = vocab[next_index]
            elif mode == "words_cos":
                top_similar, probas = list(zip(*fast_text_model.wv.most_similar([preds])))
                next_word = top_similar[sample(probas, diversity)]
            
            # add next word to queue
            start = start[1:]
            if mode == "char":
                start.append(char_idx[next_word])
            elif mode == "words_ce" or mode == "words_cos":
                start.append(fast_text_model.wv[next_word])
            
            # add next word to result
            if mode == "char":
                res += next_word
            else:
                res.append(next_word)
        print(pformat(sentence + res), "\n")
    print('='*80 + '\n')

Посмотрим на выходы моделей с разной температурой.

In [32]:
model = load_model("./models/char_lstm.h5")
infer(model, window=30, mode="char")

----- Diversity: 0.0

----- Generating with seed:
"второй третий и четвертый аргу"

второй третий и четвертый аргумент . и сказал господь моисею говоря скажи сынам израилевым в сердце своем . и сказал господь моисею говоря скажи сынам израилевым в сердце своем . и сказал господь моисею говоря скажи сынам израилев 

----- Diversity: 0.2

----- Generating with seed:
"второй третий и четвертый аргу"

второй третий и четвертый аргуым по приношению их . и сказал им иосиф и сказал ему пойди в сердце твоем . и сказал господь моисею говоря скажи ему вот я дал тебе и сказал вот я наполнил тебя и сказал вот я поставил меня . и послал 

----- Diversity: 0.4

----- Generating with seed:
"второй третий и четвертый аргу"

второй третий и четвертый аргу иоав сын иоава и поставил с ними . в этом случае в этом случае функция signal в случае успеха поскольку процесс создает текущий каталог процесс вызывает функцию pthreadmutexattrinit . и сказал господ 

----- Diversity: 0.6000000000000001

----- Genera

In [33]:
model = load_model("./models/char_bidirect.h5")
infer(model, window=30, mode="char")

----- Diversity: 0.0

----- Generating with seed:
"if n sysconfschostnamemax n ho"

if n sysconfschostnamemax n hostnamemax . и сказал господь моисею говоря объяви его и сказал ему не поступай с ним . и сказал господь моисею говоря объяви его и сказал ему не поступай с ним . и сказал господь моисею говоря объяви  

----- Diversity: 0.2

----- Generating with seed:
"if n sysconfschostnamemax n ho"

if n sysconfschostnamemax n hoего и сказал ему имя его . и сказал господь моисею говоря вот сыны израилевы подобно что поставил в себя все стана и поставил себе колена израилевы и все служащие господней . и сказал господь моисею г 

----- Diversity: 0.4

----- Generating with seed:
"if n sysconfschostnamemax n ho"

if n sysconfschostnamemax n hoоворя обратил мне господу все сии от сего дня . и сказал господь моисею говоря скажи сынам израилевым и сказал не построил господь и построил все бога и обложил его в стене и на долине божие . и сказа 

----- Diversity: 0.6000000000000001

----- Genera

In [32]:
model = load_model("./models/ce.h5")
infer(model, window=5, mode="words_ce")

----- Diversity: 0.0

----- Generating with seed:
"использование барьера include include include"

использование барьера include include include apue . и сказал господь моисею говоря объяви сынам израилевым и скажи им когда придете в землю ханаанскую в которую ты идешь чтоб овладеть ею . и сказал господь моисею говоря объяви сынам израилевым и скажи им когда придете в 

----- Diversity: 0.2

----- Generating with seed:
"использование барьера include include include"

использование барьера include include include землю ханаанскую в которую ты идешь чтоб овладеть ею . и сказал господь моисею говоря скажи сынам израилевым если кто из сынов израилевых не введете в нее и не будешь пить . и сказал господь моисею говоря объяви сынам израилевым 

----- Diversity: 0.4

----- Generating with seed:
"использование барьера include include include"

использование барьера include include include и скажи им когда придете в землю ханаанскую к иакову в землю ханаанскую и рассказали ему за то что ты не с

In [35]:
model = load_model("./models/cos.h5")
infer(model, window=5, mode="words_cos")

----- Diversity: 0.0

----- Generating with seed:
"давайте рассмотрим два конкретных случая"

давайте рассмотрим два конкретных случая нн оно лепсо придите pcchownrestricted тестя неудачу оно vo оно лечь елдад щ оно vo нн некорректно онан vo постоянно иисус щ некорректно if сильнее struct сильнее vo нее иисус nй некорректно действительно елдад щ tsys hh нн постоянно мне 

----- Diversity: 0.2

----- Generating with seed:
"давайте рассмотрим два конкретных случая"

давайте рассмотрим два конкретных случая иаир некорректно ситуации нее иаир ирад enqueuemsgstruct оставьте rif тест щ ю между щ credstruct жителям щ елдад нужным иевусеев удивительно щ херувимов некто poll некорректно my приходят ресурса египтяне struct извне from тестя iff тест аргумента досталось аморреев неявно 

----- Diversity: 0.4

----- Generating with seed:
"давайте рассмотрим два конкретных случая"

давайте рассмотрим два конкретных случая иоав ipv ресурсы щ нетрудно иевусеев мудр controlb ситуаций щ интерпретацию юa

In [36]:
model = load_model("./models/char_lstm_under.h5")
infer(model, window=30, mode="char")

----- Diversity: 0.0

----- Generating with seed:
"socket предопределенное имя се"

socket предопределенное имя семафора . и сказал господь моисею говоря возьми себе и поставил их и сказал им и сказал им и сказал им и сказал им и сказал им и сказал им и сказал им и сказал им и сказал им и сказал им и сказал им и  

----- Diversity: 0.2

----- Generating with seed:
"socket предопределенное имя се"

socket предопределенное имя сесказал им и сказал сыны израилевы сыны израилевы и сказали ему и он сказал им и послал и положил их и пошли с ним и сказал им на себе который вы поставили его и сказал вот и послал сын мой и сказал ем 

----- Diversity: 0.4

----- Generating with seed:
"socket предопределенное имя се"

socket предопределенное имя сеу господь вознесет в нем и послал и возвратился и все земли вашей в который возьми в сердце его . например программа поддерживает файл поддерживаются в случае успеха в случае ошибки int flag . в проце 

----- Diversity: 0.6000000000000001

----- Genera

In [37]:
model = load_model("./models/char_bidirect_under.h5")
infer(model, window=30, mode="char")

----- Diversity: 0.0

----- Generating with seed:
"за описанием структур в system"

за описанием структур в system . и сказал ему сын израилевы и поставил их и сказал ему и сказал господь бог твой . и сказал ему и сказал господь бог твой . и сказал ему и сказал господь бог твой . и сказал ему и сказал господь бог 

----- Diversity: 0.2

----- Generating with seed:
"за описанием структур в system"

за описанием структур в system твой . и сказал ему исполняет и сказал ему в семь старом и сказал ему и на семь своем . и сказал ему семь свое свое при не создавал их . и возвратился в стана и пошли и сказал господь бог твой и сказ 

----- Diversity: 0.4

----- Generating with seed:
"за описанием структур в system"

за описанием структур в systemал господь бог твой . и положил собрание все дня вам есть возвращает из сигнала sigset и sigalrm из системы на реального времени запускаемого процесса . и сказал господь моисею своему . и пришли сыны  

----- Diversity: 0.6000000000000001

----- Genera

По этим результатам мне кажется лучше всего выбрать темепературу 0.4-0.6 и underfit модели на символах.

Функция, возвращающая сгенерированную моделью последовательность по входной последовательности:

In [33]:
def sample_sentences(model, sentence=None, mode="char", window=30, diversity=0.3, seq_len=200):
    '''
    Returns sequences of length seq_len for each diversity
    mode: {"char", "words_ce", "words_cos"}
    '''
    vocab = alphabet if mode == "char" else words
    
    if sentence is None:
        sentence = np.random.choice(np.concatenate([code_test, testament_test]), 1)[0]
    if mode == "char":
        sentence = " ".join(sentence)
        sentence = sentence[:window]
        start = [np.zeros_like(char_idx["."]).tolist()] * (window - len(sentence))
        start.extend([char_idx[ch] for ch in sentence])
    elif mode == "words_cos" or "words_ce":
#         sentence = [w for s in sentence for w in s][:window]
        start = [np.zeros_like(fast_text_model.wv["."])] * (window - len(sentence))
        start.extend([fast_text_model.wv[w] for w in sentence])
    
    res = '' if mode == "char" else []
    if seq_len is 200:
        seq_len = 200 if mode == "char" else 40
    for i in range(seq_len):

        preds = model.predict(np.array(start)[None, :, :], verbose=0)[0]

        # sample next word
        if mode == "char" or mode == "words_ce":
            next_index = sample(preds, diversity)
            next_word = vocab[next_index]
        elif mode == "words_cos":
            top_similar, probas = list(zip(*fast_text_model.wv.most_similar([preds])))
            next_word = top_similar[sample(probas, diversity)]

        # add next word to queue
        start = start[1:]
        if mode == "char":
            start.append(char_idx[next_word])
        elif mode == "words_ce" or mode == "words_cos":
            start.append(fast_text_model.wv[next_word])

        # add next word to result
        if mode == "char":
            res += next_word
        else:
            res.append(next_word)
    print(pformat(sentence + res))

Выберем несколько предложений из теста в качестве сидов.

In [42]:
code_s = np.random.choice(np.concatenate([code_test]), 5)
testament_s = np.random.choice(np.concatenate([testament_test]), 5)

In [43]:
print(*code_s)
print()
print(*testament_s)

['целью', 'этих', 'руководств', 'было', 'повышение', 'переносимости', 'прикладных', 'программ', 'которое', 'стало', 'возможным', 'благодаря', 'простому', 'следованию', 'опубликованным', 'стандартам', '.'] ['в', 'противном', 'случае', 'мы', 'изменяем', 'реальный', 'и', 'эффективный', 'идентификаторы', 'вызовом', 'функций', 'setgid', 'и', 'setuid', '.'] ['h', 'pidt', 'tcgetsidint', 'fd', '.'] ['предыстория', 'одной', 'из', 'популярных', 'библиотек', 'функций', 'для', 'работы', 'с', 'базами', 'данных', 'в', 'unix', 'является', 'библиотека', 'dbm', '.'] ['однако', 'файл', 'должен', 'существовать', 'если', 'другой', 'процесс', 'смог', 'установить', 'на', 'него', 'блокировку', '.']

['он', 'злословил', 'меня', 'тяжким', 'злословием', 'когда', 'я', 'шел', 'в', 'маханаим', '.'] ['мы', 'умертвим', 'их', 'и', 'искореним', 'зло', 'из', 'израиля', '.'] ['хотя', 'бы', 'до', 'полуцарства', 'она', 'будет', 'исполнена', '.'] ['аврам', 'послушался', 'слов', 'сары', '.'] ['иосиф', 'подвел', 'их', 'к', '

In [44]:
test_s = [
    ['но', 'мы', 'будем', 'исходить', 'из', 'предположения', 'что', 'вызывающий', 'процесс', 
     'просто', 'не', 'пожелал', 'сбрасывать', 'содержимое', 'буферов', '.'],
    ['выполнение', 'программ', 'командной', 'оболочкой', '.'],
    ['целью', 'этих', 'руководств', 'было', 'повышение', 'переносимости', 'прикладных', 'программ', 
     'которое', 'стало', 'возможным', 'благодаря', 'простому', 'следованию', 'опубликованным', 'стандартам', '.'],
    ['однако', 'файл', 'должен', 'существовать', 'если', 'другой', 'процесс', 'смог', 'установить', 'на', 'него', 'блокировку', '.'],
    
    ['не', 'ешь', 'с', 'нею', 'квасного', '.'],
    ['но', 'я', 'отрок', 'малый', 'не', 'знаю', 'ни', 'моего', 'выхода', 'ни', 'входа', '.'],
    ['в', 'шестой', 'же', 'день', 'собрали', 'хлеба', 'вдвое', 'по', 'два', 'гомора', 'на', 'каждого', '.'],
    ['он', 'злословил', 'меня', 'тяжким', 'злословием', 'когда', 'я', 'шел', 'в', 'маханаим', '.']
]

In [59]:
model = load_model("./models/char_lstm_under.h5")
for s in test_s:
    sent = []
    i = 0
    while len(" ".join(sent + [s[i]])) <= 30:
        sent.append(s[i])
        i += 1
        if i == len(s):
            break
    print(f"T=0.4")
    sample_sentences(model, sentence=sent, mode="char", window=30, diversity=0.4)
    print(f"T=0.6")
    sample_sentences(model, sentence=sent, mode="char", window=30, diversity=0.6)
    print()

T=0.4
но мы будем исходить из состояния может получить переменную от дочернего процесса . и возвратился и сказал господь бог твой . после вызова функции read в последнем случае не получает поддержку последнего изменения процесса 
T=0.6
но мы будем исходить из нового дочерей согрешили три против сие . и пришел иудей говорил ему и сказал по себе и все возможности и в момент командной оболочки . если же поставил его из народа который вы был о рукою своем с н

T=0.4
выполнение программ командной оболочки . h int pthreadrwlocktrlecpthreadbarrierattrt devtid . например вызывает поток при программе из листинга . и поставил господь поставил себе и послал его и все израильтяне в сердце своем и ск
T=0.6
выполнение программ командной оболочки которые приложения обычно обработать файл создаются самым приостановким передача такого случая и поддерживается это последние дескрипторы процесса . когда мы можем открыть пример открытия фа

T=0.4
целью этих руководств было создана поток и обработчик сигнал

In [60]:
model = load_model("./models/char_bidirect_under.h5")
for s in test_s:
    sent = []
    i = 0
    while len(" ".join(sent + [s[i]])) <= 30:
        sent.append(s[i])
        i += 1
        if i == len(s):
            break
    print(f"T=0.4")
    sample_sentences(model, sentence=sent, mode="char", window=30, diversity=0.4)
    print(f"T=0.6")
    sample_sentences(model, sentence=sent, mode="char", window=30, diversity=0.6)
    print()

T=0.4
но мы будем исходить из сигнала sigconf . и сказал ему вот становил господь бог твой и на после во все семь дней на него и сказал им не положил что бог твой . и сказал ему по отца моего и сказал господь бог твой он будет от
T=0.6
но мы будем исходить из пользователя . h с . возвращает в случае успеха код ошибки в случае успеха в случае ошибки int sigintint signo . реальный сигнал sigint или exec в этой сигналось над сигналом и не обработки поддержив

T=0.4
выполнение программ командной строки . функция poll выполняет передачи процесса . и послал господь моисею сказали им земля так тебе . при вызове функции signal и sigalrm . в случае успеха в случае ошибки int fd struct otemax stru
T=0.6
выполнение программ командной оболочки . такое использование одну как глава так против пророков и стана . в качестве пользователя в системе . тогда бог он все пред ним пользовательскому для над строки на рис . h с . возвращает в 

T=0.4
целью этих руководств было соответствующим и поддерживает пр

In [68]:
model = load_model("./models/char_lstm_under.h5")
sample_sentences(model, sentence="придумал бог алгоритм а он".split(), mode="char", window=30, diversity=0.6)

придумал бог алгоритм а они посвятил господь соединение и не определены на серверных случая . когда работает от меня в поля struct passwd fd . вызов функции system остановить возможность вернуться в данной потоке . обокнавальн


In [77]:
sample_sentences(model, sentence="придумал бог алгоритм а он".split(), mode="char", window=30, diversity=0.4)

придумал бог алгоритм а он принесет в египет . и послал им и возвратился сын его по стражу и простирал раба моего в семьдесят первых вершины . и сказал господь моисею говоря он поставил его . и сказал господь моисею говоря пос


In [71]:
sample_sentences(model, sentence="значение аргумента".split(), mode="char", window=30, diversity=0.6)

значение аргумента control вызывает read которая управляют левиты с тебя чтобы аарон и иерусалим в пустыне до разного пред себе . и сказал господь моисею вот я повелел ему и сыновья своего против бить и возврати службу


In [74]:
sample_sentences(model, sentence="если исходный текст".split(), mode="char", window=30, diversity=0.6)

если исходный текст . и отдал господь моисею с всех городах и стал был иерусалима и приносите их и сказал аарону и ты нам у своего одного на дом свой . и дай тебе стояли который принесли народ твой из дома их да это был


In [67]:
sample_sentences(model, sentence="вызов функции".split(), mode="char", window=30, diversity=0.6)

вызов функции setprintf а управляющий терминал является одной при процессе из системы которые можно изменить оба только в структуре функции wait . тогда сказал такого господь сделал и было дело его и не повелел ем
